# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/Output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Camalú,30.8500,-116.0667,13.33,56,21,4.92,MX,1614826703
1,Busselton,-33.6500,115.3333,25.36,67,44,4.14,AU,1614826616
2,Hobart,-42.8794,147.3294,22.00,24,20,11.32,AU,1614826473
3,Rikitea,-23.1203,-134.9692,27.02,78,74,3.15,PF,1614826612
4,Ambilobe,-13.2000,49.0500,23.21,94,42,0.53,MG,1614826703


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [36]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = cities_df["Humidity"].astype(float)

In [37]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("Images/vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
first = cities_df.loc[cities_df["Max Temp"]<80,:]
second = first.loc[first["Max Temp"]>10,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Najrān,17.4924,44.1277,17.00,29,0,2.06,SA,1614826704
7,Constitución,-35.3333,-72.4167,17.59,67,0,3.85,CL,1614826704
47,Lüderitz,-26.6481,15.1594,19.79,62,0,5.65,NaN,1614826710
56,Lebu,-37.6167,-73.6500,16.03,84,0,6.97,CL,1614826711
58,Medininagar,24.0333,84.0667,25.34,16,0,2.03,IN,1614826452


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
5,Najrān,SA,17.4924,44.1277
7,Constitución,CL,-35.3333,-72.4167
47,Lüderitz,NaN,-26.6481,15.1594
56,Lebu,CL,-37.6167,-73.6500
58,Medininagar,IN,24.0333,84.0667


In [31]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 5: Najrān.
Closest hotel to Najrān at 17.4924 , 44.1277 is ماكينة صراف آلي الاهلي.
------------
Retrieving Results for Index 7: Constitución.
Closest hotel to Constitución at -35.3333 , -72.4167 is Riveradelmaule.
------------
Retrieving Results for Index 47: Lüderitz.
Closest hotel to Lüderitz at -26.6481 , 15.1594 is Kratzplatz.
------------
Retrieving Results for Index 56: Lebu.
Closest hotel to Lebu at -37.6167 , -73.65 is La Caleta Delivery.
------------
Retrieving Results for Index 58: Medininagar.
Closest hotel to Medininagar at 24.0333 , 84.0667 is Axis Bank ATM.
------------
Retrieving Results for Index 62: Doha.
Closest hotel to Doha at 25.2867 , 51.5333 is La Villa Palace Hotel.
------------
Retrieving Results for Index 71: Hurghada.
Closest hotel to Hurghada at 27.2574 , 33.8129 is Hurghada Dreams Hotel Apartment.
------------
Retrieving Results for Index 96: Hobyo.
Closest hotel to Hobyo at 5.3505 , 48.5268 is Almubarak Farsamo.
------------
Re

In [29]:
hotel_df.head()

,City,Country,Lat,Lng
5,Najrān,SA,17.4924,44.1277
7,Constitución,CL,-35.3333,-72.4167
47,Lüderitz,NaN,-26.6481,15.1594
56,Lebu,CL,-37.6167,-73.6500
58,Medininagar,IN,24.0333,84.0667


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map

# Display figure
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate1, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("Images/vacationheatmapwithhotels.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>